# Versuche für Filter-Stage

In [11]:
import pdal 
import numpy as np
import seaborn as sns
import open3d as o3d
import os

from interessant import *

In [12]:
interessant.keys()

dict_keys(['Einfach', 'Weiche', 'Ende', 'Bahnübergang', 'Bahnübergang 2', 'Zug run 14 A', 'Zug run 14 B', 'Zug run 24 A', 'Zug run 24 B', 'Gebäude', 'Gleis hohe Intensität', 'Feldweg', 'Feld', 'Weiche A', 'Weiche B', 'Zaun', 'Straße', 'Kante in Graben', 'Unterirdischer Bhf', 'Fangschiene Tunnel', 'Fangschiene Tunnel 2', 'Gleis weit abseits', 'Komische Linie', 'Kabeltöpfe', 'Güterzug', 'Güterzug Ende', 'Betondeckel', 'Bahnsteig', 'Bahnsteig Ende', 'Ding neben Gleis', 'Wände', 'Viele Gleise'])

In [13]:
run = run24
filename = interessant["Einfach"]
filename = os.path.join(run24, filename)
os.path.exists(filename)

True

In [14]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]
points.dtype # Column names and types

dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('Synthetic', 'u1'), ('KeyPoint', 'u1'), ('Withheld', 'u1'), ('Overlap', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('ScanChannel', 'u1'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])

In [15]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()
rgb = np.vstack((points['Red'], points['Green'], points['Blue'])).transpose() / 65535.0

# Offset entfernen
offset = xyz.mean(axis=0)
xyz -= offset

In [16]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(rgb)

In [18]:
o3d.visualization.draw_geometries([pcd])

## Test Filter auf Intensity

In [31]:
def intensity_filter(points, thresh=14500):
    mask = points['Intensity'] < thresh
    filtered_array = points[mask]
    print(f"Filtered out {len(points) - len(filtered_array)} points (old: {len(points)}, new: {len(filtered_array)})")
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(np.vstack((filtered_array['X'], filtered_array['Y'], filtered_array['Z'])).transpose())
    filtered_pcd.colors = o3d.utility.Vector3dVector(np.vstack((filtered_array['Red'], filtered_array['Green'], filtered_array['Blue'])).transpose() / 65535.0)
    return filtered_array, filtered_pcd


In [33]:
filtered_points, filtered_pcd = intensity_filter(points)

Filtered out 1505235 points (old: 1803127, new: 297892)


In [34]:
o3d.visualization.draw_geometries([filtered_pcd])